### Generate music using LSTM

 - code adapted from https://www.kaggle.com/code/ohseokkim/music-generation-let-s-enjoy-new-music/notebook
 - https://git.arts.ac.uk/lmccallum/STEM-4-Creatives-22-23/blob/main/STEM-Week-5-Task-Solutions.ipynb
- audio classification code adapted from https://git.arts.ac.uk/tbroad/AI-4-Media-23-24/blob/main/Week-6a-Audio-classification/01-train-audio-classifier-solution.ipynb

In [3]:
%pip install music21 numpy pandas scikit-learn tensorflow

  Using cached music21-9.1.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached chardet-5.2.0-py3-none-any.whl.metadata (3.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.8/22.8 MB 3.0 MB/s eta 0:00:0000:0100:01
Using cached chardet-5.2.0-py3-none-any.whl (199 kB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
import numpy as np
import pandas as pd
from music21 import converter, instrument, note, chord, stream, stream
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Set the path to your classical music MIDI files
dataset_path = '/Users/nixi/Desktop/Final-thesis-folder-24/classical'

def get_notes(midi_path):
    notes = []
    try:
        midi = converter.parse(midi_path)
        parts = instrument.partitionByInstrument(midi)
        if parts:
            notes_to_parse = parts.parts[0].recurse()
        else:
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    except Exception as e:
        print(f"Error processing {midi_path}: {e}")
    return notes

all_notes = []
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith('.mid'):
            file_path = os.path.join(root, file)
            print(f"Processing: {file_path}")  # Debug print
            notes = get_notes(file_path)
            all_notes.extend(notes)

print(f"Total notes processed: {len(all_notes)}")  # Debug print

# Get all unique note names
unique_notes = sorted(set(all_notes))
note_to_int = dict((note, number) for number, note in enumerate(unique_notes))

print(f"Number of unique notes: {len(unique_notes)}")  # Debug print

# Prepare sequences
sequence_length = 100
network_input = []
network_output = []

for i in range(0, len(all_notes) - sequence_length, 1):
    sequence_in = all_notes[i:i + sequence_length]
    sequence_out = all_notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)
n_vocab = len(unique_notes)

print(f"Number of sequences: {n_patterns}")  # Debug print

network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
network_input = network_input / float(n_vocab)

# Ensure network_output is not empty before converting to categorical
if network_output:
    network_output = to_categorical(network_output)
    print(f"Shape of network_output: {network_output.shape}")  # Debug print
else:
    print("Error: network_output is empty")

# The rest of your code (model definition, training, etc.) goes here

Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/mozart/mz_570_1.mid


/Users/nixi/Desktop/Final-thesis-folder-24/.conda/lib/python3.11/site-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2006 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(


Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/mozart/mz_570_2.mid
Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/mozart/mz_570_3.mid
Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/mozart/mz_545_1.mid
Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/mozart/mz_332_3.mid


/Users/nixi/Desktop/Final-thesis-folder-24/.conda/lib/python3.11/site-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2006 by Bernd Kr\xfcger'>; getting generic Instrument
  warnings.warn(


Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/mozart/mz_330_1.mid


...

Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/muss/muss_7.mid
Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/muss/muss_6.mid
Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/muss/muss_8.mid


/Users/nixi/Desktop/Final-thesis-folder-24/.conda/lib/python3.11/site-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=0, channel=None, data=b'Baba-Yaga  - Das gro\xdfe Tor von Kiew'>; getting generic Instrument
  warnings.warn(


Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/borodin/bor_ps1.mid
Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/borodin/bor_ps2.mid
Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/borodin/bor_ps3.mid
Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/borodin/bor_ps7.mid
Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/borodin/bor_ps6.mid
Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/borodin/bor_ps4.mid
Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/borodin/bor_ps5.mid
Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/brahms/brahms_opus1_4.mid
Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/brahms/brahms_opus1_2.mid
Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/brahms/br_rhap.mid


/Users/nixi/Desktop/Final-thesis-folder-24/.conda/lib/python3.11/site-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=4, channel=None, data=b'Produced 1997 by Bernd Kr\xfcger.'>; getting generic Instrument
  warnings.warn(


Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/brahms/brahms_opus1_3.mid
Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/brahms/brahms_opus1_1.mid
Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/brahms/brahms_opus117_1.mid
Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/brahms/br_im5.mid
Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/brahms/brahms_opus117_2.mid
Processing: /Users/nixi/Desktop/Final-thesis-folder-24/classical/brahms/br_im2.mid
Total notes processed: 11362
Number of unique notes: 222
Number of sequences: 11262
Shape of network_output: (11262, 222)


In [10]:
print(f"Number of notes processed: {len(all_notes)}")
print(f"Number of unique notes: {len(unique_notes)}")

Number of notes processed: 11362
Number of unique notes: 222


In [11]:
sequence_length = 100
network_input = []
network_output = []

for i in range(0, len(all_notes) - sequence_length, 1):
    sequence_in = all_notes[i:i + sequence_length]
    sequence_out = all_notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)
n_vocab = len(unique_notes)

network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
network_input = network_input / float(n_vocab)
network_output = to_categorical(network_output)

# Build and start training sequential model ( also used in emotion classification)
model = Sequential([
    LSTM(256, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True),
    Dropout(0.3),
    LSTM(512, return_sequences=True),
    Dropout(0.3),
    LSTM(256),
    Dense(256),
    Dropout(0.3),
    Dense(n_vocab, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

model.fit(network_input, network_output, epochs=50, batch_size=64)

Epoch 1/50


/Users/nixi/Desktop/Final-thesis-folder-24/.conda/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


176/176 ━━━━━━━━━━━━━━━━━━━━ 103s 577ms/step - loss: 4.8651
Epoch 2/50
176/176 ━━━━━━━━━━━━━━━━━━━━ 105s 594ms/step - loss: 4.4765
Epoch 3/50
176/176 ━━━━━━━━━━━━━━━━━━━━ 104s 591ms/step - loss: 4.4072
Epoch 4/50
176/176 ━━━━━━━━━━━━━━━━━━━━ 105s 596ms/step - loss: 4.3435
Epoch 5/50
176/176 ━━━━━━━━━━━━━━━━━━━━ 105s 598ms/step - loss: 4.2602
Epoch 6/50
176/176 ━━━━━━━━━━━━━━━━━━━━ 106s 604ms/step - loss: 4.2142
Epoch 7/50
176/176 ━━━━━━━━━━━━━━━━━━━━ 112s 635ms/step - loss: 4.1338
Epoch 8/50
176/176 ━━━━━━━━━━━━━━━━━━━━ 113s 639ms/step - loss: 4.0038
Epoch 9/50
176/176 ━━━━━━━━━━━━━━━━━━━━ 113s 640ms/step - loss: 3.9098
Epoch 10/50
176/176 ━━━━━━━━━━━━━━━━━━━━ 113s 644ms/step - loss: 3.8225
Epoch 11/50
176/176 ━━━━━━━━━━━━━━━━━━━━ 113s 643ms/step - loss: 3.6574
Epoch 12/50
176/176 ━━━━━━━━━━━━━━━━━━━━ 115s 652ms/step - loss: 3.5732
Epoch 13/50
176/176 ━━━━━━━━━━━━━━━━━━━━ 117s 665ms/step - loss: 3.5092
Epoch 14/50
176/176 ━━━━━━━━━━━━━━━━━━━━ 116s 660ms/step - loss: 3.3149
Epoch 15/50


In [16]:
def generate_notes(model, network_input, unique_notes, n_vocab, num_notes=500):
    start = np.random.randint(0, len(network_input)-1)
    int_to_note = dict((number, note) for number, note in enumerate(unique_notes))
    pattern = network_input[start]
    prediction_output = []

    for _ in range(num_notes):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
        prediction = model.predict(prediction_input, verbose=0)
        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        pattern = np.append(pattern, index)
        pattern = pattern[1:]

    return prediction_output

def create_midi(prediction_output, filename):
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp=filename)

# Generate music for each emotion
emotions = ['happy', 'sad', 'angry', 'neutral']
for emotion in emotions:
    generated_notes = generate_notes(model, network_input, unique_notes, n_vocab)
    create_midi(generated_notes, f'{emotion}_generated_music.mid')

In [18]:
X_train, X_val, y_train, y_val = train_test_split(network_input, network_output, test_size=0.2, random_state=42)

In [19]:
# Assuming you've split your data into training and validation sets
val_loss = model.evaluate(X_val, y_val)
print(f"Validation loss: {val_loss}")

71/71 ━━━━━━━━━━━━━━━━━━━━ 16s 225ms/step - loss: 0.3005
Validation loss: 0.2995627820491791


### Analyse generated music

In [20]:
def analyze_midi(filename):
    midi = converter.parse(filename)
    notes = midi.flat.notesAndRests
    
    note_counts = {}
    durations = []
    
    for element in notes:
        if isinstance(element, note.Note):
            pitch = str(element.pitch)
            if pitch in note_counts:
                note_counts[pitch] += 1
            else:
                note_counts[pitch] = 1
            durations.append(element.duration.quarterLength)
    
    avg_duration = sum(durations) / len(durations)
    most_common_note = max(note_counts, key=note_counts.get)
    
    print(f"Analysis for {filename}:")
    print(f"Total notes: {len(notes)}")
    print(f"Unique notes: {len(note_counts)}")
    print(f"Most common note: {most_common_note} (occurred {note_counts[most_common_note]} times)")
    print(f"Average note duration: {avg_duration:.2f} quarter notes")
    print("\n")

# Analyze each generated MIDI file
for emotion in emotions:
    analyze_midi(f'{emotion}_generated_music.mid')

/Users/nixi/Desktop/Final-thesis-folder-24/.conda/lib/python3.11/site-packages/music21/stream/base.py:3694: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


Analysis for happy_generated_music.mid:
Total notes: 563
Unique notes: 17
Most common note: C4 (occurred 33 times)
Average note duration: 0.87 quarter notes


Analysis for sad_generated_music.mid:
Total notes: 563
Unique notes: 17
Most common note: C4 (occurred 33 times)
Average note duration: 0.87 quarter notes


Analysis for angry_generated_music.mid:
Total notes: 563
Unique notes: 17
Most common note: C4 (occurred 33 times)
Average note duration: 0.87 quarter notes


Analysis for neutral_generated_music.mid:
Total notes: 563
Unique notes: 17
Most common note: C4 (occurred 33 times)
Average note duration: 0.87 quarter notes




### Visualize note distributions

In [ ]:
import matplotlib.pyplot as plt

def plot_note_distribution(filename):
    midi = converter.parse(filename)
    notes = [str(n.pitch) for n in midi.flat.notes if isinstance(n, note.Note)]
    
    note_counts = {}
    for note in notes:
        if note in note_counts:
            note_counts[note] += 1
        else:
            note_counts[note] = 1
    
    sorted_counts = sorted(note_counts.items(), key=lambda x: x[1], reverse=True)
    notes, counts = zip(*sorted_counts[:20])  # Top 20 notes
    
    plt.figure(figsize=(12, 6))
    plt.bar(notes, counts)
    plt.title(f'Note Distribution for {filename}')
    plt.xlabel('Notes')
    plt.ylabel('Frequency')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Plot note distribution for each generated MIDI file
for emotion in emotions:
    plot_note_distribution(f'{emotion}_generated_music.mid')

### Compare music to original dataset

In [ ]:
def compare_to_original(original_notes, generated_files):
    original_note_counts = {}
    for note in original_notes:
        if note in original_note_counts:
            original_note_counts[note] += 1
        else:
            original_note_counts[note] = 1

    print("Comparison to original dataset:")
    for file in generated_files:
        generated_midi = converter.parse(file)
        generated_notes = [str(n.pitch) for n in generated_midi.flat.notes if isinstance(n, note.Note)]
        
        generated_note_counts = {}
        for note in generated_notes:
            if note in generated_note_counts:
                generated_note_counts[note] += 1
            else:
                generated_note_counts[note] = 1

        # Calculate Jaccard similarity
        intersection = set(original_note_counts.keys()) & set(generated_note_counts.keys())
        union = set(original_note_counts.keys()) | set(generated_note_counts.keys())
        jaccard_similarity = len(intersection) / len(union)

        print(f"\nFile: {file}")
        print(f"Jaccard similarity: {jaccard_similarity:.4f}")
        print(f"Notes in original but not in generated: {len(set(original_note_counts.keys()) - set(generated_note_counts.keys()))}")
        print(f"Notes in generated but not in original: {len(set(generated_note_counts.keys()) - set(original_note_counts.keys()))}")

# Compare generated music to original dataset
generated_files = [f'{emotion}_generated_music.mid' for emotion in emotions]
compare_to_original(all_notes, generated_files)